# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from bson.decimal128 import Decimal128

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [7]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6595889474a11a2a0c91fbec'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    'BusinessName': 'Penang Flavours',
    'BusinessType': 'Cafe',
    'RatingValue': 4,
    'LocalAuthorityName': 'London',
    'PostCode': 'SW1A 1AA',
    'geocode': {'latitude': -37.787628, 'longitude': 145.131432}
}



In [10]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant_data)

In [11]:
# Check that the new restaurant was inserted
# Query the 'establishments' collection to find the newly inserted restaurant
query = {'BusinessName': 'Penang Flavours'}
result = establishments.find_one(query)

# Check if the document was found
if result:
    print("Penang Flavours data found:")
    pprint(result)
else:
    print("Penang Flavours data not found.")



Penang Flavours data found:
{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Cafe',
 'LocalAuthorityName': 'London',
 'PostCode': 'SW1A 1AA',
 'RatingValue': 4,
 '_id': ObjectId('65958977193d625a7e15ba48'),
 'geocode': {'latitude': -37.787628, 'longitude': 145.131432}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# Define the query for "Restaurant/Cafe/Canteen"
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}

# Projection to return only specific fields
projection = {'BusinessTypeID': 1, 'BusinessType': 1, '_id': 0}

# Find the document matching the query and apply the projection
result = establishments.find_one(query, projection)

# Print the result
if result:
    print("BusinessTypeID and BusinessType for 'Restaurant/Cafe/Canteen':")
    pprint(result)
else:
    print("No matching document found.")


BusinessTypeID and BusinessType for 'Restaurant/Cafe/Canteen':
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
# Define the filter for the document to update (based on BusinessName)
filter_query = {'BusinessName': 'Penang Flavours'}

# Define the update operation to set the correct BusinessTypeID
update_operation = {'$set': {'BusinessTypeID': 1}}

# Perform the update
update_result = establishments.update_one(filter_query, update_operation)

In [14]:
# Confirm that the new restaurant was updated
# Query the 'establishments' collection to find the updated restaurant
updated_query = {'BusinessName': 'Penang Flavours'}
updated_result = establishments.find_one(updated_query)

# Check if the document was found
if updated_result:
    print("Updated restaurant data:")
    pprint(updated_result)
else:
    print("Updated restaurant data not found.")


Updated restaurant data:
{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Cafe',
 'BusinessTypeID': 1,
 'LocalAuthorityName': 'London',
 'PostCode': 'SW1A 1AA',
 'RatingValue': 4,
 '_id': ObjectId('65958977193d625a7e15ba48'),
 'geocode': {'latitude': -37.787628, 'longitude': 145.131432}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
# Define the query to find documents with LocalAuthorityName as "Dover"
dover_query = {'LocalAuthorityName': 'Dover'}

# Count the number of documents with LocalAuthorityName as "Dover"
dover_count = establishments.count_documents(dover_query)

# Print the result
print(f"Number of documents with LocalAuthorityName 'Dover': {dover_count}")


Number of documents with LocalAuthorityName 'Dover': 994


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
# Define the filter to identify documents with LocalAuthorityName as "Dover"
dover_filter = {'LocalAuthorityName': 'Dover'}

# Delete all documents matching the filter
delete_result = establishments.delete_many(dover_filter)

# Print the result
print(f"Number of documents deleted: {delete_result.deleted_count}")


Number of documents deleted: 994


In [17]:
# Check if any remaining documents include Dover
# Count the number of documents with LocalAuthorityName as "Dover" after deletion
remaining_dover_count = establishments.count_documents({'LocalAuthorityName': 'Dover'})

# Print the result
print(f"Number of remaining documents with LocalAuthorityName 'Dover': {remaining_dover_count}")


Number of remaining documents with LocalAuthorityName 'Dover': 0


In [18]:
# Check that other documents remain with 'find_one'
# Find one document after deletion
remaining_document = establishments.find_one()

# Print the result
if remaining_document:
    print("Details of a remaining document:")
    pprint(remaining_document)
else:
    print("No remaining documents found.")


Details of a remaining document:
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6595889474a11a2a0c91fed1'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude
update_result = establishments.update_many(
    {
        '$or': [
            {'geocode.latitude': {'$type': 'string'}},
            {'geocode.longitude': {'$type': 'string'}}
        ]
    },
    {
        '$set': {
            'geocode.latitude': {'$toDouble': '$geocode.latitude'},
            'geocode.longitude': {'$toDouble': '$geocode.longitude'}
        }
    }
)

# Print the result
print(f"Number of documents updated: {update_result.modified_count}")

Number of documents updated: 38785


Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
# Change the data type from String to Integer for RatingValue
update_result = establishments.update_many(
    {'RatingValue': {'$type': 'string'}},
    [{'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}]
)

# Print the result
print(f"Number of documents updated for RatingValue conversion: {update_result.modified_count}")

Number of documents updated for RatingValue conversion: 34694


In [22]:
# Check that the coordinates and rating value are now numbers
converted_document = establishments.find_one()

# Print the result
if converted_document:
    print("Details of a document after data type conversion:")
    pprint(converted_document)
else:
    print("No documents found.")


Details of a document after data type conversion:
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6595889474a11a2a0c91fed1'),
 'geocode': {'latitude': {'$toDouble': '$geocode.latitude'},
             'longitude': {'$toDouble': '$geocode.longitude'}},
 'links': [{'href': 'h